In [ ]:
# uv venv --python=python3.10
# source .venv/bin/activate

In [ ]:
# Entonces la idea nueva es, identificar la interacción o las interacciones más 
# largas de los Alertantes, obtener palabras clave, con esas palabras clave pedir 
# a un LLM que genere preguntas (esas preguntas deben relacionarse con las 
# palabras clave y no deben contener NERs) y luego comparar semánticamente 
# las preguntas generadas por el LLM con la base de datos de preguntas reales, 
# para lanzarlas como opción para que diga el operador.

In [ ]:
# Primero se tiene que armar el dataset
# Se me ocurren dos ideas, como solo vamos a trabajar la siguiente pregunta
# que debe decir el operador y se va a consultar al LLM que genere una posible pregunta
# incluyendo un tipo de COT en el prompt con los metadatos que tienen las interacciones
# y luego comparar esa pregunta semánticamente con la base de preguntas normalizadas para dar unas opciones
# al operador, y finalmente para la evaluación comparar tal vez la similitud semántica entre la opción escogida
# o la pregunta que ha generado el LLM contra la pregunta actual que ha dicho el operador.

# Entonces podría ser una primera prueba enviar todo el contexto previo de la conversación hasta antes de
# lo que dice el alertante en el prompt.

# Y el otro experimento solo enviando lo que dice el alertante, sin contexto previo.

# Entonces los datasets del primer experimento se van a guardar en v6/datasets/v1
# y los del segundo experimento en v6/datasets/v1


In [ ]:
# Primero voy a armar el módulo de recuperación por similitud semántica
# no voy a ocupar Milvus, Pinecone, etc. porque no son muchos registros
# y no hay mucho tiempo xd.

In [1]:
# como al cargar el LLM gpt oss de 20b se necesitan las dos primeras gpus
# casi a full, para el resto voy a ocupar la 3era gráfica

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2" 

In [2]:
# Primero voy a unificar en una sola todas las preguntas generales de los
# operadores
import pandas as pd

df_pregunta_confirmación = pd.read_csv("../vllm_experiments/reduccion_interacciones_operador/reduccion_pregunta/reduccion_confirmacion/v1/final_pregunta_confirmacion.csv")
df_pregunta_contacto = pd.read_csv("../vllm_experiments/reduccion_interacciones_operador/reduccion_pregunta/reduccion_contacto/v1/final_pregunta_contacto.csv")
df_pregunta_heridos = pd.read_csv("../vllm_experiments/reduccion_interacciones_operador/reduccion_pregunta/reduccion_heridos/v1/final_pregunta_heridos.csv")
df_pregunta_localizacion = pd.read_csv("../vllm_experiments/reduccion_interacciones_operador/reduccion_pregunta/reduccion_localizacion/v1/final_pregunta_localizacion.csv")
df_pregunta_otro = pd.read_csv("../vllm_experiments/reduccion_interacciones_operador/reduccion_pregunta/reduccion_otro/v1/final_pregunta_otro.csv")
df_pregunta_tipo_incidente = pd.read_csv("../vllm_experiments/reduccion_interacciones_operador/reduccion_pregunta/reduccion_tipo_incidente/v1/final_pregunta_tipo_incidente.csv")

concat_df = pd.concat([df_pregunta_confirmación, df_pregunta_contacto, df_pregunta_heridos,
                       df_pregunta_localizacion, df_pregunta_otro, df_pregunta_tipo_incidente])

concat_df = concat_df.drop_duplicates(subset=['canonical_questions'])
print(concat_df.shape)

(72, 1)


In [3]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('intfloat/multilingual-e5-large',
                            device="cuda")

questions = concat_df["canonical_questions"].to_list()
formatted_questions = ["query: " + question for question in questions]
corpus_embeddings = model.encode(formatted_questions, normalize_embeddings=True)
# corpus_embeddings = corpus_embeddings.to("cuda")
# corpus_embeddings = util.normalize_embeddings(corpus_embeddings)

In [ ]:
# para pruebas
query = "query: gas"
query_embeddings = model.encode(query, normalize_embeddings=True)
# query_embeddings = query_embeddings.to("cuda")
# query_embeddings = util.normalize_embeddings(query_embeddings)

hits = util.semantic_search(query_embeddings, corpus_embeddings, top_k=3)
top_hits = hits[0]

for hit in top_hits:
    corpus_id = hit["corpus_id"]
    score = hit["score"]
    texto = questions[corpus_id]
    print(f"Score: {score:.4f} | Pregunta: {texto}")

In [4]:
# función para extraer las n preguntas parecidas

def retrieve_real_questions(question):
    formatted_query = f"query: {question}"
    query_embedding = model.encode(formatted_query, normalize_embeddings=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    top_hits = hits[0]
    related_real_questions = []
    scores = []
    
    for hit in top_hits:
        corpus_id = hit["corpus_id"]
        score = hit["score"]
        text = questions[corpus_id]
        related_real_questions.append(text)
        scores.append(score)
    
    return related_real_questions, scores

In [1]:
# cargar los datasets iniciales
# como no vamos a entrenar ningún modelo
# solo vamos a formatear el dataset principal
# para hacer la inferencia
import pandas as pd

df = pd.read_excel("../vllm_experiments/processed_llm_output_anonimized_clean.xlsx")

# Primer experimento, enviando contexto previo al prompt

## Preparando dataset

In [2]:
# para generar el dataset del primer experimento
from transformers import pipeline

pipe_keywords = pipeline(
    "text2text-generation", 
    model="UDA-LIDI/barto_emergency_multi_purpose",
    device=0
)

def get_keywords(text):
    return pipe_keywords("Extrae las palabras clave de la emergencia: " + text)

def _create_context_sample(actor, text):
    return f"ACTOR: {actor} TEXT: {text}"

def make_context(row):
    return _create_context_sample(
        row.ACTOR,
        row.ATA_TEXTO_TAGGED,
    )

def context_slide_window(tmp_df):
    aux_context = ""
    context = []
    samples = []
    keywords = []    
    labels = []
 
    rows = list(tmp_df.itertuples())

    for i in range(len(rows) - 1):
        current = rows[i]
        next_turn = rows[i + 1]

        if i > 0:
            aux_context += make_context(rows[i - 1]) + " \n "
            # context.append(make_context(rows[i - 1]))

        if current.COMBINED == "Alertante-Descripción-tipo_incidente":
            context.append(aux_context)
            samples.append(make_context(current))
            keywords.append(get_keywords(current.ATA_TEXTO_TAGGED)[0]["generated_text"])
            labels.append(make_context(next_turn))
            break
            
    return context, samples, keywords, labels

def prepare_datasets_for_training(dataframe):
    unique_ids = dataframe["TRA_ID"].unique()
    final_context = []
    final_samples = []
    final_keywords = []    
    final_labels = []
    for id in unique_ids:
        temp_df = dataframe[dataframe["TRA_ID"] == id].copy()
        temp_df["COMBINED"] = temp_df["ACTOR"] + "-" + temp_df["DIALOG_ACT"] + "-" + temp_df["SUBACT"]
        aux_combined = temp_df["COMBINED"].to_list()
        if "Alertante-Descripción-tipo_incidente" in aux_combined:
            train_c, train_s, train_k, train_labs = context_slide_window(temp_df)
            final_context += train_c
            final_samples += train_s
            final_keywords += train_k
            final_labels += train_labs
        
    return pd.DataFrame({
        "prepared_context": final_context,
        "prepared_samples": final_samples,
        "prepared_keywords": final_keywords,
        "prepared_labels": final_labels
    })

Device set to use cuda:0


In [4]:
prepared_dataset = prepare_datasets_for_training(df)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [5]:
print(prepared_dataset.shape)

(172, 4)


In [ ]:
# por qué hay menos registros que las 250 conversaciones iniciales
# esto es porque yo filtré para encontrar únicamente las interacciones
# donde esa interacción la haya dicho un alertante y fue etiquetada
# como una descripción y un subacto de "tipo_incidente"

# si bien todas las conversaciones deberían tener al menos una interacción
# de ese tipo, revisando algunos ejemplos, hay alertantes que para describir
# lo que está pasando describen directamente heridos o describen síntomas
# y fueron etiquetadas con otro subacto, como descripción de síntomas
# o descripción de heridos.

In [6]:
prepared_dataset.iloc[0]

prepared_context     ACTOR: Operador TEXT: emergencia \n ACTOR: Ale...
prepared_samples     ACTOR: Alertante TEXT: esta un carro parqueado...
prepared_keywords                   tirar piedras rompieron parabrisas
prepared_labels                    ACTOR: Operador TEXT: le agredieron
Name: 0, dtype: object

In [7]:
prepared_dataset.iloc[0]["prepared_context"]

'ACTOR: Operador TEXT: emergencia \n ACTOR: Alertante TEXT: buenos dias hagame un favor me puede mandar un patrullero \n ACTOR: Operador TEXT: que sucede señore que le sucede \n '

In [8]:
prepared_dataset.iloc[0]["prepared_samples"]

'ACTOR: Alertante TEXT: esta un carro parqueado y un par de jovenes comenzaron a tirar piedras y me rompieron el parabrisas y estan aqui yo no se quienes son'

In [9]:
prepared_dataset.iloc[0]["prepared_keywords"]

'tirar piedras rompieron parabrisas'

In [10]:
prepared_dataset.iloc[0]["prepared_labels"]

'ACTOR: Operador TEXT: le agredieron'

In [ ]:
# guardar el dataset preparado
# prepared_dataset.to_csv("datasets/v1/prepared_dataset.csv", index=False)

## Inferencia

In [ ]:
# Ahora si a realizar inferencia armando un COT para solicitar al LLM

In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("datasets/v1/prepared_dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   prepared_context   172 non-null    object
 1   prepared_samples   172 non-null    object
 2   prepared_keywords  172 non-null    object
 3   prepared_labels    172 non-null    object
dtypes: object(4)
memory usage: 5.5+ KB


In [6]:
# este es el cliente del gpt oss que está corriendo como
# servicio en el hpc
from openai import OpenAI

system_prompt = """You are an expert emergency call operator assistant.
Your task is to determine the most appropriate next question that an emergency operator should ask.
You must reason based on operational emergency protocols.
Do not include names, addresses, or personal identifiable information.
Output only one concise and natural question in Spanish.
Reasoning: high
"""

client = OpenAI(base_url="http://localhost:8010/v1", api_key="EMPTY")

def get_llm_answer(user_prompt):
    
    result = client.chat.completions.create(
        model="openai/gpt-oss-20b",
        messages=[
            {
                "role": "system",
                "content": system_prompt  
            },
            {
                "role": "user",
                "content": user_prompt
            },
        ],
    )
    
    return (result.choices[0].message.content)

In [7]:
def extract_operator_text(label):
    return label.split("TEXT:")[1].strip()

def build_CoT_prompt(row):
    return f"""
You are given an emergency call transcript.

Previous context:
{row['prepared_context']}

Current alertant description:
{row['prepared_samples']}

Extracted keywords:
{row['prepared_keywords']}

Instructions:
- Identify the type of incident described.
- Determine what critical information is missing.
- Formulate the most relevant operational question an emergency operator should ask next.
- The question must be generic and reusable.
- The final output must be a single question in natural Spanish.
"""


In [ ]:
df["label_operator_text"] = df["prepared_labels"].progress_apply(extract_operator_text)

In [9]:
df["llm_generated_question"] = df.progress_apply(
    lambda row: get_llm_answer(build_CoT_prompt(row)),
    axis=1
)

In [ ]:
# def retrieve_top3_questions(llm_question):
#     retrieved = retrieve_real_questions(llm_question)
#     top3 = retrieved[:3]
    
#     texts = [q for _, q in top3]
#     scores = [score for score, _ in top3]
    
#     return texts, scores


In [ ]:
df[["retrieved_questions", "retrieved_scores"]] = df.progress_apply(
    lambda row: pd.Series(retrieve_real_questions(row["llm_generated_question"])),
    axis=1
)

In [12]:
def semantic_similarity_top3(label_text, retrieved_questions):
    texts = [label_text] + retrieved_questions
    # se debe formatear para este modelo, el E5 large
    formatted = ["query: " + t for t in texts]
    embeddings = model.encode(
        formatted,
        normalize_embeddings=True
    )
    
    similarities = model.similarity(
        embeddings[0:1],
        embeddings[1:]
    )

    return similarities[0].tolist()

In [ ]:
df["semantic_sim_top_vs_label"] = df.progress_apply(
    lambda row: semantic_similarity_top3(
        row["label_operator_text"],
        row["retrieved_questions"]
    ),
    axis=1
)

In [ ]:
df["best_hit_similarity"] = df["semantic_sim_top_vs_label"].progress_apply(max)

In [16]:
df.to_excel(
    "results/experiment_v1_llm_retrieval.xlsx",
    index=False
)